In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from wordcloud import WordCloud
from gensim import corpora, models

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils

In [ ]:
# Initialize DataLoader
sl = SlackDataLoader('../Anonymized_B6SlackExport_25Nov23/anonymized/')

### Columns we can get from a slack message<br>

message_type, message_content, sender_id, time_sent, message_distribution, time_thread_start, reply_count, reply_user_count, time_thread_end, reply_users

From a single slack message, we can get <br>

1. The message<br>
2. Type (message, file, link, etc)<br>
3. The sender_id (assigned by slack)<br>
4. The time the message was sent<br>
5. The team (i don't know what that is now)<br>
6. The type of the message (broadcast message, inhouse, just messgae)<br>
7. The thread the message generated (from here we can go):<br>
    7.1 Text/content of the message<br>
    7.2 The thread time of the message<br>
    7.3 The thread count (reply count)<br>
    7.4 The number of user that reply the message (count of users that participated in the thread)<br>
    7.5 The time the last thread message was sent <br>
    7.6 The users that participated in the thread (their ids are stored as well)<br>

## Insight Extraction

Below are some useful questions to answer. Feel free to explore to answer other interesting questions that may be of help to get insight about student's behaviour, need, and future performance 

In [ ]:
# which user has the highest number of reply counts?
# Assuming DataFrame is named 'df'
df = combined_data 
user_with_highest_replies = df[df['reply_count'] == df['reply_count'].max()]['sender_name'].iloc[0]
print(f"The user with the highest number of reply counts is: {user_with_highest_replies}")


In [ ]:
# Visualize reply counts per user per channel
import seaborn as sns
import matplotlib.pyplot as plt
df = combined_data 

# Assuming your DataFrame is named 'df'
plt.figure(figsize=(15, 7.5))
sns.barplot(x='sender_name', y='reply_count', hue='channel', data=df, ci=None)
plt.title('Reply Counts per User per Channel', size=15, fontweight='bold')
plt.xlabel('Sender Name', size=12); plt.ylabel('Reply Count', size=12)
plt.xticks(rotation=45, ha='right', size=10)
plt.legend(title='Channel', title_fontsize='12', loc='upper right')
plt.show()

In [ ]:
# which user has the highest number of reply counts?# what is the time range of the day that most messages are sent?

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
df['msg_sent_time'] = pd.to_datetime(df['msg_sent_time'])
df['hour_of_day'] = df['msg_sent_time'].dt.hour

# Plotting the distribution of messages across hours
plt.figure(figsize=(15, 7.5))
sns.countplot(x='hour_of_day', data=df)
plt.title('Distribution of Messages Across Hours of the Day', size=15, fontweight='bold')
plt.xlabel('Hour of Day', size=12)
plt.ylabel('Message Count', size=12)
plt.show()


In [11]:
# what kind of messages are replied faster than others?
import pandas as pd
df = combined_data 

def analyze_response_time(df):
    # Convert 'tm_thread_end' and 'msg_sent_time' to datetime format if they are in string format
    df['tm_thread_end'] = pd.to_datetime(df['tm_thread_end'], errors='coerce')
    df['msg_sent_time'] = pd.to_datetime(df['msg_sent_time'], errors='coerce')

    # Drop rows with missing values in 'tm_thread_end' or 'msg_sent_time'
    df = df.dropna(subset=['tm_thread_end', 'msg_sent_time'])

    # Calculate response time
    df['response_time'] = df['tm_thread_end'] - df['msg_sent_time']

    # Categorize messages based on keywords in the content
    df['message_category'] = df['msg_content'].apply(lambda x: 'urgent' if 'urgent' in x.lower() else 'normal')

    # Aggregate data based on message categories
    result = df.groupby('message_category')['response_time'].mean()

    # Print or return the result
    print(result)
    return result


analyze_response_time(df)

In [15]:
# Relationship between # of messages and # of reactions

In [16]:
# Classify messages into different categories such as questions, answers, comments, etc.
import re

def classify_message(message):
    # Define patterns for different message categories
    question_pattern = re.compile(r'\b(?:how|what|when|where|why)\b', flags=re.IGNORECASE)
    answer_pattern = re.compile(r'\b(?:here is|solution)\b', flags=re.IGNORECASE)
    comment_pattern = re.compile(r'\b(?:nice|great|explanation)\b', flags=re.IGNORECASE)

    # Check for patterns and assign categories
    if re.search(question_pattern, message):
        return 'Question'
    elif re.search(answer_pattern, message):
        return 'Answer'
    elif re.search(comment_pattern, message):
        return 'Comment'
    else:
        return 'Other'

# Example usage
df['message_category'] = df['message_content'].apply(classify_message)
print(df)


In [17]:
# Which users got the most reactions?

In [33]:
# Model topics mentioned in the channel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Assuming df['msg_content'] contains the messages
documents = df['msg_content'].dropna().values.astype('U')

# Create a document-term matrix using CountVectorizer
vectorizer = CountVectorizer(max_features=1000, stop_words='english')
dtm = vectorizer.fit_transform(documents)

# Apply Latent Dirichlet Allocation
lda = LatentDirichletAllocation(n_components=5, random_state=42)
topics = lda.fit_transform(dtm)

# Display the top words for each topic
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    top_words_idx = topic.argsort()[:-10-1:-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(f"Topic #{topic_idx + 1}: {', '.join(top_words)}")

# Assign the dominant topic to each document
df['dominant_topic'] = topics.argmax(axis=1)

In [ ]:
# What are the topics that got the most reactions?

### Harder questions to look into

In [ ]:
# Based on messages, reactions, references shared, and other relevant data such as classification of questions into techical question, comment, answer, aorder stu the python, statistics, and sql skill level of a user?